In [1]:
import pandas as pd
import numpy as np
import os
import wandb
import joblib

# Sınıflandırma modeli ve metrikleri için
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Ham veriyi yükle (ilk 10 dosya ile)
DATA_PATH = 'data'
FILES_TO_READ = [f"vm_cpu_readings-file-{i}-of-195.csv.gz" for i in range(1, 6)]
COLUMN_NAMES = ['timestamp', 'vmId', 'min_cpu', 'max_cpu', 'avg_cpu']
df_list = []
for file_name in FILES_TO_READ:
    full_path = os.path.join(DATA_PATH, file_name)
    if os.path.exists(full_path):
        temp_df = pd.read_csv(full_path, header=None, names=COLUMN_NAMES, compression='gzip')
        df_list.append(temp_df)
df_raw = pd.concat(df_list, ignore_index=True)

# Veriyi etiketle
anomaly_threshold = 90
df_raw['etiket'] = (df_raw['avg_cpu'] >= anomaly_threshold).astype(int)

# Özellik (Feature) ve Hedef (Target) Belirle
X = df_raw[['min_cpu', 'max_cpu', 'avg_cpu']]
y = df_raw['etiket']

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("Veri setleri anormallik tespiti için hazır.")
print("Etiket Dağılımı (Tüm Veri):")
print(y.value_counts(normalize=True))

Veri setleri anormallik tespiti için hazır.
Etiket Dağılımı (Tüm Veri):
etiket
0    0.992132
1    0.007868
Name: proportion, dtype: float64


In [3]:
def run_detection_experiment(config):
    # 1. WandB Deneyini Başlat
    run = wandb.init(project="bites-cpu-detection", config=config, reinit=True)
    
    # 2. Config'e göre modeli seç ve oluştur
    if config['model_type'] == 'DecisionTree':
        model = DecisionTreeClassifier(
            max_depth=config['max_depth'],
            random_state=42
        )
    elif config['model_type'] == 'RandomForest':
        model = RandomForestClassifier(
            n_estimators=config['n_estimators'],
            max_depth=config['max_depth'],
            random_state=42,
            n_jobs=-1
        )
    else:
        raise ValueError("Desteklenmeyen model tipi")

    # 3. Modeli eğit
    print(f"\n--- Deney Başladı: {run.name} ({config['model_type']}) ---")
    model.fit(X_train, y_train)
    print("Eğitim tamamlandı.")

    # Her deneye özel bir dosya adı oluşturalım
    model_filename = f"model_path/{config['model_type']}_{run.name}.joblib"
    joblib.dump(model, model_filename)
    print(f"Model '{model_filename}' olarak diske kaydedildi.")
    
    # Modeli bir "artifact" olarak WandB'ye de kaydedelim
    wandb.save(model_filename)
    # --- BİTTİ ---

    # 4. Tahmin yap
    y_pred = model.predict(X_test)
    y_probas = model.predict_proba(X_test)
    
    # 5. Metrikleri hesapla ve WandB'ye logla
    report = classification_report(y_test, y_pred, output_dict=True, target_names=['Normal', 'Anormal'])
    
    wandb.log({
        "accuracy": report['accuracy'],
        "precision_anormal": report['Anormal']['precision'],
        "recall_anormal": report['Anormal']['recall'],
        "f1_score_anormal": report['Anormal']['f1-score']
    })
    
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(
        probs=y_probas,
        y_true=y_test.to_numpy(),
        class_names=['Normal', 'Anormal']
    )})
    
    print(f"'{config['model_type']}' için metrikler WandB'ye loglandı.")
    
    # 6. Deneyi bitir
    run.finish()

In [4]:
# Denenecek modeller ve hiperparametreleri
configs = [
    {
        'model_type': 'DecisionTree', 
        'max_depth': 5
    },
    {
        'model_type': 'RandomForest', 
        'n_estimators': 100,
        'max_depth': 10
    }
]

# Tüm konfigürasyonları sırayla dene
for cfg in configs:
    run_detection_experiment(cfg)

wandb: Currently logged in as: melihak635442 (melihak635442-bites) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.



--- Deney Başladı: stellar-butterfly-7 (DecisionTree) ---
Eğitim tamamlandı.
Model 'model_path/DecisionTree_stellar-butterfly-7.joblib' olarak diske kaydedildi.
'DecisionTree' için metrikler WandB'ye loglandı.


accuracy,▁
f1_score_anormal,▁
precision_anormal,▁
recall_anormal,▁
accuracy,1.0
f1_score_anormal,1.0
precision_anormal,1
recall_anormal,0.99999



--- Deney Başladı: serene-grass-8 (RandomForest) ---
Eğitim tamamlandı.
Model 'model_path/RandomForest_serene-grass-8.joblib' olarak diske kaydedildi.
'RandomForest' için metrikler WandB'ye loglandı.


accuracy,▁
f1_score_anormal,▁
precision_anormal,▁
recall_anormal,▁
accuracy,1.0
f1_score_anormal,1.0
precision_anormal,1
recall_anormal,0.99999
